In [ ]:
import yt
import numpy as np
import plotly.graph_objects as go
from plotly.io import write_image
import os
import sys
import pandas as pd


In [ ]:
#import yt

pf = "/home/u0890475/Downloads/plt59250"   # <- directory, not a single file
ds = yt.load(pf)
cg = ds.covering_grid(
    level=0,
    left_edge=ds.domain_left_edge,
    dims=ds.domain_dimensions
)

# cell size at level 0
dx = ds.domain_width / ds.domain_dimensions  # [dx, dy, dz]
dx0, dy0, dz0 = map(float, dx)
print("dx,dy,dz:", dx0, dy0, dz0)
print(ds.field_list)

In [ ]:
import re
def pick_first_field(ds, candidates):
    for name in candidates:
        ft = ("boxlib", name)
        if ft in ds.field_list:
            return ft
    return None

def find_species_fields(ds):
    # Common PeleLMeX patterns: Y(H2), Y_O2, YH2, massfrac_H2, MassFraction_H2, ...
    out = []
    for ft in ds.field_list:
        if ft[0] != "boxlib":
            continue
        n = ft[1]
        if re.match(r"^Y\(.+\)$", n):
            out.append(ft)
        elif re.match(r"^Y[_A-Za-z0-9]+$", n):
            out.append(ft)
        elif "massfrac" in n.lower() or "massfraction" in n.lower():
            out.append(ft)
    # de-dupe preserve order
    seen = set()
    uniq = []
    for f in out:
        if f not in seen:
            seen.add(f)
            uniq.append(f)
    return uniq

def amr_cell_centers(ds):
    xs, ys, zs = [], [], []
    lvls = []
    dxs, dys, dzs = [], [], []

    for g in ds.index.grids:
        le = g.LeftEdge.to("code_length").d
        dds = g.dds.to("code_length").d
        nx, ny, nz = g.ActiveDimensions.astype(int)
        lvl = int(g.Level)

        x1 = le[0] + (np.arange(nx) + 0.5) * dds[0]
        y1 = le[1] + (np.arange(ny) + 0.5) * dds[1]
        z1 = le[2] + (np.arange(nz) + 0.5) * dds[2]
        X, Y, Z = np.meshgrid(x1, y1, z1, indexing="ij")

        n = X.size
        xs.append(X.ravel()); ys.append(Y.ravel()); zs.append(Z.ravel())
        lvls.append(np.full(n, lvl, dtype=np.int16))
        dxs.append(np.full(n, dds[0], dtype=np.float64))
        dys.append(np.full(n, dds[1], dtype=np.float64))
        dzs.append(np.full(n, dds[2], dtype=np.float64))

    return (np.concatenate(xs), np.concatenate(ys), np.concatenate(zs),
            np.concatenate(lvls), np.concatenate(dxs), np.concatenate(dys), np.concatenate(dzs))

def amr_cells_to_df(ds,
                    vel_fields=None,
                    heat_release_field=None,
                    include_species=True):
    # --- choose velocity fields ---
    if vel_fields is None:
        ux = pick_first_field(ds, ["x_velocity", "xvel", "u", "velx", "x_vel"])
        uy = pick_first_field(ds, ["y_velocity", "yvel", "v", "vely", "y_vel"])
        uz = pick_first_field(ds, ["z_velocity", "zvel", "w", "velz", "z_vel"])
        vel_fields = (ux, uy, uz)

    ux_f, uy_f, uz_f = vel_fields
    if ux_f is None or uy_f is None or uz_f is None:
        raise RuntimeError("Could not find velocity fields. Inspect ds.field_list and set vel_fields manually.")

    # --- choose heat release field (optional) ---
    if heat_release_field is None:
        heat_release_field = pick_first_field(ds, [
            "heat_release", "HeatRelease", "HeatReleaseRate",
            "HRR", "heatRelease", "heat_release_rate"
        ])
    hr_f = heat_release_field  # may be None

    # --- species fields ---
    species_fs = find_species_fields(ds) if include_species else []

    # --- base coords (build from your function) ---
    x, y, z, level, dx, dy, dz = amr_cell_centers(ds)

    # We must extract fields in the SAME grid loop order to align with x/y/z
    u_all, v_all, w_all = [], [], []
    hr_all = [] if hr_f is not None else None
    spec_all = {f[1]: [] for f in species_fs}

    for g in ds.index.grids:
        u_all.append(g[ux_f].d.ravel())
        v_all.append(g[uy_f].d.ravel())
        w_all.append(g[uz_f].d.ravel())

        if hr_f is not None:
            hr_all.append(g[hr_f].d.ravel())

        for f in species_fs:
            spec_all[f[1]].append(g[f].d.ravel())

    data = {
        "x": x, "y": y, "z": z,
        "level": level,
        "dx": dx, "dy": dy, "dz": dz,
        "u": np.concatenate(u_all),
        "v": np.concatenate(v_all),
        "w": np.concatenate(w_all),
    }

    if hr_f is not None:
        data["heat_release"] = np.concatenate(hr_all)

    for name, chunks in spec_all.items():
        data[name] = np.concatenate(chunks)

    return pd.DataFrame(data), {"ux": ux_f, "uy": uy_f, "uz": uz_f, "heat_release": hr_f, "species": species_fs}

df, used = amr_cells_to_df(ds)

print("Used fields:", used)
print("df shape:", df.shape)
print(df.head())

In [ ]:
import yt
import numpy as np
import pandas as pd
import re

# --------------------
# User inputs
# --------------------
pf = "/home/u0890475/Downloads/plt59250"  
#pf = "/path/to/plt00050"            # PeleLMeX plotfile DIRECTORY
temp_field = ("boxlib", "temp")     # change if needed
ux_field = ("boxlib", "x_velocity")   # <-- change here if yours differ
uy_field = ("boxlib", "y_velocity")   # <-- change here
uz_field = ("boxlib", "z_velocity")
Tcut = 400.0
xmax = 0.07                         # in code_length units (same units as ds.domain)

# If HRR autodetect fails, set explicitly, e.g. ("boxlib","heat_release_rate")
hrr_field_override = None

# --------------------
# Load
# --------------------
ds = yt.load(pf)
ad = ds.all_data()

# --------------------
# Helpers
# --------------------
def find_first_boxlib_field_by_regex(patterns):
    for ft in ds.field_list:
        if ft[0] != "boxlib":
            continue
        name = ft[1]
        for pat in patterns:
            if re.search(pat, name, flags=re.IGNORECASE):
                return ft
    return None

def find_species_fields():
    out = []
    for ft in ds.field_list:
        if ft[0] != "boxlib":
            continue
        n = ft[1]
        if re.match(r"^Y\(.+\)$", n):  # Y(H2)
            out.append(ft); continue
        if re.match(r"^Y[_A-Za-z0-9]+$", n):  # Y_H2, YO2, YH2
            if not re.search(r"dot|rate", n, re.IGNORECASE):
                out.append(ft); continue
        if re.search(r"mass\s*frac|massfrac|MassFraction", n, re.IGNORECASE):
            out.append(ft); continue

    seen = set()
    out2 = []
    for f in out:
        if f not in seen:
            out2.append(f)
            seen.add(f)
    return out2

def pick_amr_level_field():
    candidates = [
        ("index", "grid_level"),
        ("index", "level"),
        ("index", "amr_level"),
        ("index", "grid_level_id"),
    ]
    for f in candidates:
        if f in ds.derived_field_list or f in ds.field_list:
            return f
    for ft in ds.derived_field_list:
        if ft[0] == "index" and re.search(r"level", ft[1], re.IGNORECASE):
            return ft
    for ft in ds.field_list:
        if ft[0] == "index" and re.search(r"level", ft[1], re.IGNORECASE):
            return ft
    return None

# --------------------
# Required fields check
# --------------------
if temp_field not in ds.field_list and temp_field not in ds.derived_field_list:
    raise RuntimeError(f"temp_field {temp_field} not found. Search ds.field_list for temperature name.")

level_field = pick_amr_level_field()
if level_field is None:
    raise RuntimeError("Could not find an AMR level field (e.g. ('index','grid_level')).")

# Heat release rate field
if hrr_field_override is not None:
    hrr_field = hrr_field_override
else:
    hrr_field = find_first_boxlib_field_by_regex([
        r"heat[_\s-]*release", r"\bHRR\b", r"qdot", r"heat[_\s-]*rate", r"heat[_\s-]*source"
    ])

if hrr_field is None or (hrr_field not in ds.field_list and hrr_field not in ds.derived_field_list):
    raise RuntimeError(
        "Could not auto-detect HRR field. Set hrr_field_override = ('boxlib','<exact_name>').\n"
        "Tip: print([f for f in ds.field_list if 'heat' in f[1].lower() or 'hrr' in f[1].lower() or 'qdot' in f[1].lower()])"
    )

# Species (mass fractions)
species_fields = find_species_fields()
if len(species_fields) == 0:
    print("WARNING: No mass-fraction fields auto-detected (continuing with coords + HRR + AMR level only).")

# --------------------
# Build mask: T > 400K and x < 0.07
# --------------------
T = ad[temp_field].d
x = ad[("index", "x")].to("code_length").d

mask = (T > Tcut) & (x < xmax)

n = int(mask.sum())
print(f"Cells with T>{Tcut} and x<{xmax}: {n}")
if n == 0:
    raise RuntimeError("No cells matched the mask")

# --------------------
# Extract arrays for masked region
# --------------------
y = ad[("index", "y")].to("code_length").d
z = ad[("index", "z")].to("code_length").d
ux_all = ad[ux_field].d
uy_all = ad[uy_field].d
uz_all = ad[uz_field].d

lvl = ad[level_field].d.astype(np.int32)
hrr = ad[hrr_field].d

data = {
    "x": x[mask],
    "y": y[mask],
    "z": z[mask],
    "T": T[mask],
    "u": ux_all[mask],
    "v": uy_all[mask],
    "w": uz_all[mask],
    "amr_level": lvl[mask],
    hrr_field[1]: hrr[mask],
}

for f in species_fields:
    data[f[1]] = ad[f].d[mask]

df = pd.DataFrame(data)

print("DataFrame shape:", df.shape)
print("Columns (first 15):", df.columns.tolist()[:15])

# Save (Parquet is strongly recommended for large selections)
df.to_parquet("cells_Tgt400_xlt0p07.parquet", index=False)
# df.to_csv("cells_Tgt400_xlt0p07.csv", index=False)

print("Wrote cells_Tgt400_xlt0p07.parquet")

In [ ]:
idx1 = df['x'].idxmin()
z_f1  = df.iloc[idx1,2]
df_filter1  = df[df["z"]==z_f1]
df_filter1.columns

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,5))
plt.xlim(0.065, 0.0695)
plt.ylim(0.005,0.01)
sc = plt.scatter(df_filter1["x"], df_filter1["y"],
                 c=df_filter1["Y(OH)"],
                 s=5,              # point size
                 cmap='Reds')   # good for combustion

plt.colorbar(sc, label="Y(OH)")
plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.title("OH Mass Fraction Field")
plt.gca().set_aspect('equal', adjustable='box')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))

strm = ax.streamplot(
    Xi, Yi, Ui, Vi,
    density=2.0,       # ⭐ MORE streamlines
    linewidth=0.8,
    arrowsize=0.4,     # ⭐ SHORTER arrows
    maxlength=0.5      # ⭐ prevents long lines
)

ax.set_xlim(0.065, 0.0695)
ax.set_ylim(0.005, 0.01)
ax.set_aspect("equal", adjustable="box")

ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
ax.set_title("Velocity Streamlines")

plt.tight_layout()
plt.show()